<a href="https://colab.research.google.com/github/priyanshu7466/Python/blob/main/Deep_Learning/Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as  pd
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/py/master/DeepLearningML/6_gradient_descent/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=30)

In [5]:
len(X_train)

22

In [6]:
len(X_test)

6

In [7]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [8]:
X_train_scaled

,age,affordibility
8,0.62,1
6,0.55,0
23,0.45,1
4,0.46,1
11,0.28,1
21,0.26,0
18,0.19,0
19,0.18,1
1,0.25,0
16,0.25,0


In [9]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)
model.fit(X_train_scaled,y_train,epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 11ms/step - loss: 0.5110 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5109 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5109 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5109 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5108 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5108 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5107 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5107 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5107 - accuracy: 0.7273
Epoch 2510/5000
1/1 [==========================

In [10]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 132ms/step


array([[0.7946565 ],
       [0.44487095],
       [0.75160444],
       [0.38521892],
       [0.649106  ],
       [0.43275386]], dtype=float32)

In [11]:
y_test

15    1
26    0
24    1
10    0
22    1
0     0
Name: bought_insurance, dtype: int64

In [13]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 163ms/step - loss: 0.4316 - accuracy: 1.0000


[0.43159255385398865, 1.0]

In [14]:
coef, intercept = model.get_weights()
coef, intercept

(array([[4.9207563],
        [1.615373 ]], dtype=float32),
 array([-2.9685636], dtype=float32))

In [18]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [19]:
X_test

,age,affordibility
15,55,1
26,23,1
24,50,1
10,18,1
22,40,1
0,22,1


In [20]:
def prediction_function(age,affordibility):
  weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)

prediction_function(0.55,1)

0.7946564916491242

In [21]:
prediction_function(0.23,1)

0.4448709242526104

In [22]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [23]:
def log_loss(y_true,y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [34]:
def gradient_descent(age,affordibility,y_true,epochs,loss_threshold):
  w1 = w2 = 1
  bias = 0
  rate = 0.5
  n = len(age)
  for i in range(epochs):
    weighted_sum = w1*age + w2*affordibility + bias
    y_predicted = sigmoid_numpy(weighted_sum)
    loss = log_loss(y_true,y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))

    bias_d = np.mean(y_predicted-y_true)
    w1 = w1 - rate*w1d
    w2 = w2 - rate*w2d
    bias = bias - rate*bias_d

    print(f'Epoch: {i}, w1: {w1}, w2: {w2}, bias: {bias}, loss: {loss}')
    if loss<=loss_threshold:
      break
  return w1,w2,bias

In [35]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000,0.4460)

Epoch: 0, w1: 0.9747919051747271, w2: 0.9652926117314453, bias: -0.10930683402229285, loss: 0.6826999650906861
Epoch: 1, w1: 0.954962518674094, w2: 0.9379637996799426, bias: -0.2053536380408363, loss: 0.6569452967137227
Epoch: 2, w1: 0.9400305031469988, w2: 0.9174549172580183, bias: -0.28938364461011523, loss: 0.6376411012464347
Epoch: 3, w1: 0.9294510661660982, w2: 0.9030593532275806, bias: -0.3627658356262566, loss: 0.623310338414833
Epoch: 4, w1: 0.9226645382905516, w2: 0.8940050901814327, bias: -0.4268824051221337, loss: 0.6126955254246821
Epoch: 5, w1: 0.9191319930176327, w2: 0.8895196555221259, bias: -0.4830486626072018, loss: 0.6047889557128929
Epoch: 6, w1: 0.9183574769295736, w2: 0.8888739294027335, bias: -0.532464878752072, loss: 0.5988196592886467
Epoch: 7, w1: 0.9198991628223949, w2: 0.8914071037748496, bias: -0.5761937920714841, loss: 0.5942181789012493
Epoch: 8, w1: 0.9233725910876591, w2: 0.8965374287756476, bias: -0.6151560055324818, loss: 0.5905748656585167
Epoch: 9, w

(4.897073586109803, 1.685808560079705, -3.026677509959364)

In [36]:
coef, intercept

(array([[4.9207563],
        [1.615373 ]], dtype=float32),
 array([-2.9685636], dtype=float32))